In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing #label encoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics #classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Build a Gaussian Classifier
model = GaussianNB()
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2)
import warnings

In [ ]:
df=pd.read_csv("spam.csv",encoding="latin-1")

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4",],axis=1,inplace=True)

In [ ]:
df['length'] = df['v2'].apply(len)
df.head()

In [ ]:
df["length"]

In [ ]:
df.rename(columns={"v2" : "message"},inplace=True)
df.rename(columns={"v1" : "class"},inplace=True)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder =LabelEncoder()
df['class']=encoder.fit_transform(df['class'])

In [ ]:
df= df.drop_duplicates()

In [ ]:
df

# EDA

In [ ]:
plt.pie(df['class'].value_counts(),labels=['not spam','spam'],autopct='%0.2f')
plt.show()

In [ ]:
import nltk
#nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
df['message'].apply(lambda x: nltk.word_tokenize(x))

df['num_words']=df['message'].apply(lambda x:len(nltk.word_tokenize(x)))
df.sample(5)

In [ ]:
df['num_sentences']=df['message'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
df[df['class']==0][['length','num_words','num_sentences']].describe()

In [ ]:
df[df['class']==1][['length','num_words','num_sentences']].describe()

In [ ]:
#for characters
plt.figure(figsize=(12,6))
sns.histplot(df[df['class']==0]['length'],color='green')
sns.histplot(df[df['class']==1]['length'],color = 'red')
plt.legend(labels=["Ham","Spam"])
plt.show()

In [ ]:
#for words
plt.figure(figsize=(12,6))
sns.histplot(df[df['class']==0]['num_words'],color='green')
sns.histplot(df[df['class']==1]['num_words'],color='red')
plt.legend(labels=["Ham","Spam"])
plt.show()

In [ ]:
#plotting a heatmap for the correlation
sns.heatmap(df.corr(numeric_only=True),annot=True)
plt.show()

In [ ]:
def text_transform(message):
    message=message.lower() #change to lowercase
    message=nltk.word_tokenize(message)
    y=[]
    for i in message:
        if i.isalnum():
            y.append(i)

        y.clear()

    #for checking punctuations and stopwords
    for i in message:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    message=y[:]
    y.clear()

    #now stemming function
    for i in message:
        y.append(ps.stem(i))

    #return y  --> returns as list
    return " ".join(y)

In [ ]:
# Removing stop words and punctuations
#nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
len(stopwords.words('english'))

#now for punctuation
import string
string.punctuation
# stemming
from nltk.stem.porter import PorterStemmer
ps =PorterStemmer()

In [ ]:
df['transformed_msg']=df['message'].apply(text_transform)

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
wc=WordCloud(width=500,height=500,min_font_size=10,background_color='white')
spam_wc=wc.generate(df[df['class']==1]['transformed_msg'].str.cat(sep=""))
plt.figure(figsize=(18,12))
plt.imshow(spam_wc)
plt.show()

In [ ]:
# Removing stop words and punctuations
#nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
len(stopwords.words('english'))

#now for punctuation
import string
string.punctuation
# stemming
from nltk.stem.porter import PorterStemmer
ps =PorterStemmer()

In [ ]:
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
df['transformed_msg']=df['message'].apply(text_transform)

In [ ]:
from wordcloud import WordCloud
wc=WordCloud(width=500,height=500,min_font_size=10,background_color='white')
spam_wc=wc.generate(df[df['class']==1]['transformed_msg'].str.cat(sep=""))

plt.figure(figsize=(18,12))
plt.imshow(spam_wc)

In [ ]:
ham_wc=wc.generate(df[df['class']==0]['transformed_msg'].str.cat(sep=""))
plt.figure(figsize=(18,12))
plt.imshow(ham_wc)

In [ ]:
spam_corpus=[]
for msg in df[df['class']==1]['transformed_msg'].tolist():
    for word in msg.split():
        spam_corpus.append(word)
from collections import Counter
word_counts = Counter(spam_corpus)
most_common_words = word_counts.most_common(30)
word_df = pd.DataFrame(most_common_words, columns=['Word', 'Count'])
plt.figure(figsize=(18,12))
sns.barplot(x='Word', y='Count', data=word_df)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=3000)
X=tfidf.fit_transform(df['transformed_msg']).toarray()
y=df['class'].values
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

In [ ]:
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB()

In [ ]:
gnb.fit(X_train,y_train)
y_pred1= gnb.predict(X_test)
print('Accuracy score of Gaussian NB is: ',accuracy_score(y_test,y_pred1))
print('Confusion Matrix of Guassian NB is: ',confusion_matrix(y_test,y_pred1))
print('Precision score of the Gaussian NB is',precision_score(y_test,y_pred1))

In [ ]:
mnb.fit(X_train,y_train)
y_pred2=mnb.predict(X_test)
print('Accuracy score of Multinomial NB is: ',accuracy_score(y_test,y_pred2))
print('Confusion Matrix of Multinomial NB is: ',confusion_matrix(y_test,y_pred2))
print('Precision score of the Multinomial NB is',precision_score(y_test,y_pred2))

In [ ]:
bnb.fit(X_train,y_train)
y_pred3=bnb.predict(X_test)
print('Accuracy score of Bernoulli NB is: ',accuracy_score(y_test,y_pred3))
print('Confusion Matrix of Bernoulli NB is: ',confusion_matrix(y_test,y_pred3))
print('Precision score of the Bernoulli NB is',precision_score(y_test,y_pred3))

# Better Visualization

In [ ]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(mnb,open('model.pkl','wb'))